In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.autograd import Variable
import torch.nn.functional as F

from torchvision import datasets, transforms
from utils import build_dataset, flops_to_string, params_to_string
from tqdm import tqdm_notebook

import time
from tensorboardX import SummaryWriter
writer = SummaryWriter(f'runs/MobileNet_{int(time.time())}')

from thop import profile
from nets.MobileNet_CIFAR import MobileNet_CIFAR

In [2]:
flops, params = profile(MobileNet_CIFAR().cpu(), input_size=(1, 3, 32,32))
print ('Flops: {}'.format(flops_to_string(flops)))
print ('Params: {}'.format(params_to_string(params)))

Flops: 14.07 MMac
Params: 121.61k


In [3]:
class Solver(object):
    def __init__(self, model, dataset='MNIST', n_epochs=100, lr=0.001):
        self.n_epochs = n_epochs
        
        self.train_loader, self.test_loader = build_dataset(dataset, './data', batch_size = 300)
        
        self.image_dim = {'MNIST': 28*28, 'CIFAR10': 3*32*32}[dataset]
                
        self.net = model().cuda()
        
        self.loss_fn = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.net.parameters(), lr=lr)
                
    def train(self):
        self.net.train()
        best_loss = 999
        for epoch_i in tqdm_notebook(range(self.n_epochs)):
            epoch_loss = 0
            for batch_idx, (images, labels) in enumerate(self.train_loader):
                images = Variable(images).cuda()
                labels = Variable(labels).cuda()
                
                logits = self.net(images)
                
                loss = self.loss_fn(logits, labels)
                
                total_loss = loss

                self.optimizer.zero_grad()
                total_loss.backward()

                self.optimizer.step()
                
                epoch_loss += float(loss.data)    
            epoch_loss /= len(self.train_loader.dataset)
            # save best model
            if best_loss > epoch_loss:
                best_loss = epoch_loss
                torch.save(self.net.state_dict(), 'models/MobileNet_CIFAR.pth')
                
            print(f'Epoch {epoch_i} | loss: {epoch_loss:.8f}')
            
            niter = epoch_i*len(self.train_loader)+batch_idx
            writer.add_scalar('Train/Loss', loss.data[0], niter)
            self.evaluate(epoch_i)
            
        
        self.net.load_state_dict(torch.load(PATH))
        print (f'Best Loss {best_loss} | Best')
        self.evaluate(epoch_i+1)
    
    def evaluate(self, epoch_i = None):
        total = 0
        correct = 0
        self.net.eval()
        for batch_idx, (images, labels) in enumerate(self.test_loader):
            images = Variable(images).cuda()

            logits = self.net(images.cuda())
            
            _, predicted = torch.max(logits.data, 1)
            
            total += labels.size(0)
            correct += (predicted.cpu() == labels).sum()
        if epoch_i is not None:
            writer.add_scalar('Test/Acc@1', (100 * correct / total), epoch_i)
        print(f'Accuracy: {100 * correct / total:.3f}%')

In [ ]:
standard_solver = Solver(MobileNet_CIFAR, 'CIFAR10')

standard_solver.train()

Files already downloaded and verified
Files already downloaded and verified


Epoch 0 | loss: 0.00613735


/home/denis/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Accuracy: 40.000%
Epoch 1 | loss: 0.00564003
Accuracy: 43.000%
Epoch 2 | loss: 0.00471466
Accuracy: 51.000%
Epoch 3 | loss: 0.00428784
Accuracy: 53.000%
Epoch 4 | loss: 0.00396135
Accuracy: 55.000%
Epoch 5 | loss: 0.00372937
Accuracy: 59.000%
Epoch 6 | loss: 0.00356571
Accuracy: 60.000%
Epoch 7 | loss: 0.00334226
Accuracy: 61.000%
Epoch 8 | loss: 0.00322550
Accuracy: 65.000%
